# Vraisemblance

## Quelques aspects techniques avant de commencer

Il est très important de faire tourner la section ci-dessous, même si d'un point de vue pratique elle ne fait que préparer les instruments dont nous allons nous servir plus tard dans le document.


In [8]:
# library utilisée pour faire le graphique
# install.packages("ggplot2") # installation, à faire une seule fois
library(ggplot2) # chargement

# couleur pour le graphique
darkred  = "#8B0000"   


### Exemple avec la binomiale

In [10]:
dbinom(x=3, size = 10, prob =0.1)

[1] 0.05739563

In [12]:
dbinom(x=3, size = 10, prob =0.3)

[1] 0.2668279

In [14]:
dbinom(x=3, size = 10, prob =0.9)

[1] 8.748e-06